# Gaussian Process Regression in Python

<a href="https://en.wikipedia.org/wiki/Gaussian_process" target="_blank">Gaussian process</a> models are one of the less well known machine learning algorithms as compared to more popular ones such as tree based models or perceptron based models. This is unfortunate as Gaussian process models are relatively straight forward to understand while being able to model relatively complex systems. In this post we will explore how to perform Gaussian process regression using Python.

The published version on Towards Data Science <a href="https://towardsdatascience.com/gaussian-process-models-7ebce1feb83d" target="_blank">can be found here</a>!

## Gaussian Processes

Gaussian process models assume that the value of an observed target <i>yₙ</i> has the form:

<i>yₙ</i> = <i></i>f(<b>x</b><i>ₙ</i>) + <i>eₙ</i>,

where <i>f</i>(<b>x</b><i>ₙ</i>) is some function giving rise to the observed targets, <b>x</b><i>ₙ</i> is the <i>n</i>th row of a set of <i>φ</i> inputs <b>x</b> = [<i>x</i>₁, <i>x</i>₂, … <i>xᵩ</i>]ᵀ, and <i>eₙ</i> is independent Gaussian noise. The conditional probability of observing <i>yₙ</i> given <i>f</i>(<b>x</b><i>ₙ</i>) is the normal distribution:

<i>p</i>(<i>yₙ</i>|<i>f</i>(<b>x</b><i>ₙ</i>)) = <i>N</i>(<i>yₙ</i>|<i>f</i>(<b>x</b><i>ₙ</i>), <i>σ</i>),

where <i>σ</i> is the standard deviation of <i>eₙ</i>. As the noise is assumed to be independent for each sample, the joint probability distribution of <i>φ</i> observed target values <i>y</i> = [<i>y</i>₁, <i>y</i>₂, … <i>yᵩ</i>]ᵀ conditioned on <i>φ</i> values of <i>f</i>(<b>x</b>)=[<i>f</i>(<b>x</b>₁), <i>f</i>(<b>x</b>₂), … <i>f</i>(<b>x</b><i>ᵩ</i>)]ᵀ is defined to be the normal distribution:

<i>p</i>(<b>y</b>|<i>f</i>(<b>x</b>)) = <i>N</i>(<b>y</b>|<i>f</i>(<b>x</b>), <b>σ</b>),

where <b>σ</b> = <i>σ</i><b>I</b> is a diagonal matrix of size <i>φ</i>×<i>φ</i>.

In order to make predictions on <b>y</b>, we need to determine the marginal probability distribution <i>p</i>(<b>y</b>). This probability distribution can be obtained by marginalizing the conditional distribution <i>p</i>(<b>y</b>|<i>f</i>(<b>x</b>)) over the distribution <i>p</i>(<i>f</i>(<b>x</b>)) using the integral:

<i>p</i>(<b>y</b>) = ∫ <i>p</i>(<b>y</b>|<i>f</i>(<b>x</b>))·<i>p</i>(<i>f</i>(<b>x</b>)) d<i>f</i>(<i>x</i>).

The distribution <i>p</i>(<i>f</i>(<b>x</b>)) is defined to be a Gaussian distribution with a mean of 0 and covariance kernel matrix <b>K</b> of size <i>φ</i>×<i>φ</i>:

<i>p</i>(<i>f</i>(<b>x</b>)) = <i>N</i>(<i>f</i>(<b>x</b>)|<b>0</b>, <b>K</b>).

The covariance matrix <b>K</b> is composed of distances between two rows in <b>x</b>, and assumes that similar inputs should give rise to similar target values in <b>y</b>. Each element in the matrix <b>K</b> is computed as:

<b>K</b>[<i>n</i>, <i>m</i>] = <i>k</i>(<b>x</b><i>ₙ</i>, <b>x</b><i>ₘ</i>),

where <i>k</i> is some function to be defined later. Using the equation for <i>p</i>(<i>f</i>(<b>x</b>)) above, we can perform the integral involved in <i>p</i>(<b>y</b>) to obtain the solution:

<i>p</i>(<b>y</b>) = ∫ <i>p</i>(<b>y</b>|<i>f</i>(<b>x</b>))·<i>p</i>(<i>f</i>(<b>x</b>)) d<i>f</i>(<b>x</b>)
= ∫ <i>N</i>(<b>y</b>|<i>f</i>(<b>x</b>), <b>σ</b>)·<i>N</i>(<i>f</i>(<b>x</b>)|<b>0</b>, <b>K</b>) d<i>f</i>(<b>x</b>)
= <i>N</i>(<b>y</b>|<b>0</b>, <b>C</b>).

where the resulting covariance matrix has the form: <b>C</b> = <b>K</b>+<b>σ</b> = <b>K</b>+σ<b>I</b>. Therefore, each element in <b>C</b> can be written as: <b>C</b>[<i>n</i>, <i>m</i>] = <i>k</i>(<b>x</b><i>ₙ</i>, <b>x</b><i>ₘ</i>) + <i>σδₙₘ</i>.

## Gaussian Process Kernels

Various covariance kernel functions for <i>k</i> can be used, such as the quadratic exponential kernel, also known as the radial basis function (RBF) kernel, or the periodic kernel. The kernel is calculated from pairs of samples (<b>x</b><i>ₙ</i>, <b>x</b><i>ₘ</i>) in <b>x</b>:

<i>k</i>(<b>x</b><i>ₙ</i>, <b>x</b><i>ₘ</i>) = exp(-||<b>x</b><i>ₙ</i> - <b>x</b><i>ₘ</i>||²/2<i>L</i>²),

where <i>L</i> is a kernel hyper parameter which we set to 1 for convenience's sake.

## Gaussian Process Predictions

Therefore, for <i>φ</i> observations of the target <b>y</b> = [<i>y</i>₁, <i>y</i>₂, … <i>yᵩ</i>]ᵀ corresponding to a set of <i>φ</i> inputs <b>x</b> = [<b>x</b>₁, <b>x</b>₂, … <b>x</b><i>ᵩ</i>]ᵀ, we want to predict the value of <i>yᵩ</i>₊₁ corresponding to some new input <b>x</b><i>ᵩ</i>₊₁. This step requires us to determine the parameters (i.e. mean and covariance) of the probability distribution <i>p</i>(<i>yᵩ</i>₊₁|<i>y</i>).

In order to determine the parameters of <i>p</i>(<i>yᵩ</i>₊₁|<i>y</i>), we start from the distribution <i>p</i>(<b>y</b>'), where <b>y</b>' = [<i>y</i>₁, <i>y</i>₂, … <i>yᵩ</i>, <i>yᵩ</i>₊₁]ᵀ is a vector of length <i>φ</i>+1. From the solution for <i>p</i>(<i>y</i>) above, we obtain for <i>p</i>(<b>y</b>') the corresponding solution:

<i>p</i>(<b>y</b>') = <i>N</i>(<b>y</b>'|<b>0</b>, <b>C</b>'),

where the new covariance matrix <b>C</b>' of size <i>φ</i>+1×<i>φ</i>+1 has the structure:

<b>C</b>' = [[<b>C</b> , <b>k</b>],

……[<b>k</b>ᵀ, <i>c</i>]],

where <b>C</b> = <b>K</b>+<b>σ</b> is the original <i>φ</i>×<i>φ</i> covariance matrix from above, <b>k</b> is a vector of length <i>φ</i> with elements given by: <b>k</b>[<i>n</i>] = <i>k</i>(<b>x</b><i>ₙ</i>, <b>x</b><i>ᵩ</i>₊₁), and <i>c</i> is a scalar containing the covariance of <b>x</b><i>ᵩ</i>₊₁ with itself: <i>c</i> = <i>k</i>(<b>x</b><i>ᵩ</i>₊₁, <b>x</b><i>ᵩ</i>₊₁)+<i>σ</i>.

As mentioned earlier, Gaussian processes are one of the few machine learning models that have an analytical solution obtained from conditional probability as follows.

If we have some vector r of normally distributed <i>N</i>(<b>r</b>|<b>μ</b>, <b>Σ</b>) random variables, partitioned into two sub-vectors of arbitrary lengths: <b>r</b> = [<b>r</b><i>ᵤ</i>, <b>r</b><i>ᵥ</i>]ᵀ, then for the conditional distribution <i>p</i>(rᵤ|rᵥ) the mean μ(<b>r</b><i>ᵤ</i>|<b>r</b><i>ᵥ</i>) and covariance <b?Σ</b>(<b>r</b><i>ᵤ</i>|<b>r</b><i>ᵥ</i>) are given by:

<b>μ</b>(<b>r</b><i>ᵤ</i>|<b>r</b><i>ᵥ</i>) = <b>μ</b><i>ᵤ</i> + <b>Σ</b><i>ᵤᵥ</i><b>Σ</b><i>ᵥᵥ</i>⁻¹(<b>r</b><i>ᵥ</i> - <b>μ</b><i>ᵥ</i>),

<b>Σ</b>(<b>r</b><i>ᵤ</i>|<b>r</b><i>ᵥ</i>) = <b>Σ</b><i>ᵤᵤ</i> - <b>Σ</b><i>ᵤᵥ</i><b>Σ</b><i>ᵥᵥ</i>⁻¹<b>Σ</b><i>ᵥᵤ</i>,

where <b>μ</b><i>ᵤ</i>/<b>μ</b><i>ᵥ</i> is the vector containing the means of the elements of <b>r</b><i>ᵤ</i>/<b>r</b><i>ᵥ</i> and <b>Σ</b><i>ᵤᵥ</i> is the covariance of the elements of <b>r</b><i>ᵤ</i> and <b>r</b><i>ᵥ</i>.

For our case, <b>r</b><i>ᵤ</i> corresponds to the new observation while <b>r</b><i>ᵥ</i>, corresponds to the old set of <i>φ</i> observations. Hence the covariance between the old and new observations <b>Σ</b><i>ᵥᵤ</i> is the vector <b>k</b> with elements <b>k</b>[<i>n</i>] = <b>k</b>(<b>x</b><i>ₙ</i>, <b>x</b><i>ᵩ</i>₊₁), the covariance of the old observations <b>Σ</b><i>ᵥᵥ</i>⁻¹ is the matrix <b>C</b> with elements <b>C</b>[<i>n</i>, <i>m</i>] = <i>k</i>(<b>x</b><i>ₙ</i>, <b>x</b><i>ₘ</i>) + <i>σδₙₘ</i>, the covariance of the new observations <b>Σ</b><i>ᵤᵤ</i> is the scalar <i>c</i> = <i>k</i>(<b>x</b><i>ᵩ</i>₊₁, <b>x</b><i>ᵩ</i>₊₁)+<i>σ</i>, and <b>r</b><i>ᵥ</i> is the set of <i>φ</i> old observations <b>y</b>. According to our definitions above <b>μ</b><i>ᵤ</i>=<b>μ</b><i>ᵥ</i>= <b>0</b>.

Putting everything together, the conditional probability distribution <i>p</i>(<i>yᵩ</i>₊₁|<b>y</b>) is a Gaussian distribution with mean and variance given by:

<i>μ</i> = <b>k</b>ᵀ<b>C</b>⁻¹<b>y</b>,

<i>s</i>² = <i>c</i> - <b>k</b>ᵀ<b>C</b>⁻¹<b>k</b>.

## Implementing Gaussian Models in Python

The analytical solutions above can be easily implemented in Python as follows!

In [1]:
import numpy as np

def RBF_kernel(x, y, l = 1.0):
    K = np.exp(-np.linalg.norm(x - y)**2 / (2 * l**2))
    return K

def make_RBF_kernel(X, l = 1.0):
    K = np.zeros([len(X), len(X)])
    for i in range(len(X)):
        for j in range(len(X)):
            K[i, j] = RBF_kernel(X[i], X[j], l)
    return K

The prediction result <i>μ</i> = <b>k</b>ᵀ<b>C</b>⁻¹<b>y</b> is calculated using the following function.

In [2]:
def gaussian_process_predict_mean(X_data, y_data, X_pred):
    rbf_kernel = make_RBF_kernel(np.vstack([X_data, X_pred]))
    K = rbf_kernel[:len(X_data), :len(X_data)]
    k = rbf_kernel[:len(X), -1]
    y_pred = np.dot(np.dot(k, np.linalg.inv(K)), y_data)
    return y_pred

Now all we need to do is to create some test data to try these functions out:

In [3]:
# Training data x and y:
x = np.array([0, 1, 2])
y = np.pi * x + np.exp(1)
X = x.reshape(-1, 1)

# New input feature to predict:
X_new = np.array([0.5])

# Calculate and print the predicted value of new y:
mean_pred = gaussian_process_predict_mean(X, y, X_new)
print("mean predict :{}".format(mean_pred))

mean predict :3.9328785636239916


As a sanity check, we can use `GaussianProcessRegressor` from `sklearn` to check our results!

In [4]:
from sklearn.gaussian_process import GaussianProcessRegressor
gpr = GaussianProcessRegressor()
gpr.fit(X, y)
print("sklearn pred: {}".format(gpr.predict(X_new.reshape(-1, 1))))

sklearn pred: [3.93287856]


This ends our demonstration of how Gaussian process models work.

***

## References
1. <a href="https://www.springer.com/gp/book/9780387310732" target="_blank">Pattern Recognition and Machine Learning</a>, C. M. Bishop, pp 303-313.
2. https://scikit-learn.org/stable/modules/gaussian_process.html
3. https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html